In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from deap import creator, base, tools, algorithms
from scoop import futures
from sklearn import linear_model, datasets
import random
import numpy
from scipy import interpolate
from sklearn.linear_model import LogisticRegression
# import matplotlib.pyplot as plt



In [2]:
# Read in data from CSV
dfData = pd.read_csv('../data/Wednesday-workingHours-withoutInfinity-Sampled.pcap_ISCX.csv')

print(dfData.groupby(' Label').size())

 Label
0    5000
1    5500
2    5500
3    5499
4    5796
dtype: int64


In [3]:
# change dfData column order
column = list(dfData.columns.values)
column.remove(' Destination IP')
column.remove(' Timestamp')
column.insert(2, ' Destination IP')
column.insert(3, ' Timestamp')

dfData = dfData[column]

In [4]:
# Encode the classification labels to numbers
# Get classes and one hot encoded feature vectors
le = LabelEncoder()
le.fit(dfData[' Label'])
allClasses = le.transform(dfData[' Label'])
allFeatures_full = dfData.drop([' Label'], axis=1)
# omit the first 4 features
allFeatures = allFeatures_full.iloc[:,4:]

In [5]:
import numpy as np
#normalize
from sklearn.preprocessing import normalize

# allFeatures = normalize(allFeatures, axis=1, copy=True, return_norm=False)

In [6]:
# allClasses = le.transform(dfData[' Label'])
# allFeatures = dfData.drop([' Label'], axis=1)

In [7]:
# Form training, test, and validation sets
X_trainAndTest, X_validation, y_trainAndTest, y_validation = train_test_split(allFeatures, allClasses, test_size=0.20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_trainAndTest, y_trainAndTest, test_size=0.20, random_state=42)

In [8]:
# import numpy as np
np.shape(allFeatures)

(27295, 80)

In [18]:
# Feature subset fitness function
def getFitness(individual, X_train, X_test, y_train, y_test):

	# Parse our feature columns that we don't use
	# Apply one hot encoding to the features
	cols = [index for index in range(len(individual)) if individual[index] == 0]
	print(type(cols),np.shape(cols))
	X_trainParsed = X_train.drop(X_train.columns[cols], axis=1)
	X_trainOhFeatures = pd.get_dummies(X_trainParsed)
# 	print(X_trainOhFeatures)
	X_testParsed = X_test.drop(X_test.columns[cols], axis=1)
	X_testOhFeatures = pd.get_dummies(X_testParsed)
    
    # Remove any columns that aren't in both the training and test sets
	sharedFeatures = set(X_trainOhFeatures.columns) & set(X_testOhFeatures.columns)
	removeFromTrain = set(X_trainOhFeatures.columns) - sharedFeatures
	removeFromTest = set(X_testOhFeatures.columns) - sharedFeatures
	X_trainOhFeatures = X_trainOhFeatures.drop(list(removeFromTrain), axis=1)
	X_testOhFeatures = X_testOhFeatures.drop(list(removeFromTest), axis=1)
    
# 	print(np.shape(X_trainOhFeatures), np.shape(X_testOhFeatures))

	# Apply logistic regression on the data, and calculate accuracy
	clf = LogisticRegression()
# 	clf = svm.SVC()
	clf.fit(X_trainOhFeatures, y_train)
	predictions = clf.predict(X_testOhFeatures)
	accuracy = accuracy_score(y_test, predictions)

	# Return calculated accuracy as fitness
	return (accuracy,)


In [20]:
#========DEAP GLOBAL VARIABLES (viewable by SCOOP)========

# Create Individual
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Create Toolbox
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, len(allFeatures.columns) - 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Continue filling toolbox...
toolbox.register("evaluate", getFitness, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


/scratch/kc3172/py3tf/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/scratch/kc3172/py3tf/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [21]:
def getHof():

	# Initialize variables to use eaSimple
	numPop = 100
	numGen = 10
	pop = toolbox.population(n=numPop)
	hof = tools.HallOfFame(numPop * numGen)
	stats = tools.Statistics(lambda ind: ind.fitness.values)
	stats.register("avg", numpy.mean)
	stats.register("std", numpy.std)
	stats.register("min", numpy.min)
	stats.register("max", numpy.max)

	# Launch genetic algorithm
	pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=numGen, stats=stats, halloffame=hof, verbose=True)

	# Return the hall of fame
	return hof


In [22]:
def getMetrics(hof):

	# Get list of percentiles in the hall of fame
	percentileList = [i / (len(hof) - 1) for i in range(len(hof))]
	
	# Gather fitness data from each percentile
	testAccuracyList = []
	validationAccuracyList = []
	individualList = []
	for individual in hof:
		testAccuracy = individual.fitness.values
		validationAccuracy = getFitness(individual, X_trainAndTest, X_validation, y_trainAndTest, y_validation)
		testAccuracyList.append(testAccuracy[0])
		validationAccuracyList.append(validationAccuracy[0])
		individualList.append(individual)
	testAccuracyList.reverse()
	validationAccuracyList.reverse()
	return testAccuracyList, validationAccuracyList, individualList, percentileList


In [23]:
# X_train = X_train[:6400]
# X_test = X_test[:1600]
# y_train = y_train[:6400]
# y_test = y_test[:1600]
# X_trainAndTest = X_trainAndTest[:8000]
# X_validation = X_validation[:2000]
# y_trainAndTest = y_trainAndTest[:8000]
# y_validation = y_validation[:2000]

In [24]:
np.shape(X_test)

(4368, 80)

In [25]:
'''
	First, we will apply logistic regression using all the features to acquire a baseline accuracy.
'''

individual = [1 for i in range(len(allFeatures.columns))]
testAccuracy = getFitness(individual, X_train, X_test, y_train, y_test)
validationAccuracy = getFitness(individual, X_trainAndTest, X_validation, y_trainAndTest, y_validation)
print('\nTest accuracy with all features: \t' + str(testAccuracy[0]))
print('Validation accuracy with all features: \t' + str(validationAccuracy[0]) + '\n')

<class 'list'> (0,)
<class 'list'> (0,)

Test accuracy with all features: 	0.8841575091575091
Validation accuracy with all features: 	0.889540208829456



In [26]:
# cols

In [27]:
'''
	Now, we will apply a genetic algorithm to choose a subset of features that gives a better accuracy than the baseline.
'''
hof = getHof()
testAccuracyList, validationAccuracyList, individualList, percentileList = getMetrics(hof)

<class 'list'> (35,)
<class 'list'> (38,)
<class 'list'> (38,)
<class 'list'> (32,)
<class 'list'> (37,)
<class 'list'> (42,)
<class 'list'> (38,)
<class 'list'> (40,)
<class 'list'> (29,)
<class 'list'> (34,)
<class 'list'> (45,)
<class 'list'> (33,)
<class 'list'> (38,)
<class 'list'> (37,)
<class 'list'> (45,)
<class 'list'> (42,)
<class 'list'> (44,)
<class 'list'> (38,)
<class 'list'> (33,)
<class 'list'> (42,)
<class 'list'> (38,)
<class 'list'> (44,)
<class 'list'> (45,)
<class 'list'> (43,)
<class 'list'> (37,)
<class 'list'> (44,)
<class 'list'> (42,)
<class 'list'> (26,)
<class 'list'> (34,)
<class 'list'> (54,)
<class 'list'> (34,)
<class 'list'> (43,)
<class 'list'> (41,)
<class 'list'> (43,)
<class 'list'> (36,)
<class 'list'> (42,)
<class 'list'> (39,)
<class 'list'> (38,)
<class 'list'> (37,)
<class 'list'> (35,)
<class 'list'> (42,)
<class 'list'> (33,)
<class 'list'> (38,)
<class 'list'> (40,)
<class 'list'> (37,)
<class 'list'> (38,)
<class 'list'> (34,)
<class 'list'

<class 'list'> (31,)
<class 'list'> (34,)
<class 'list'> (26,)
<class 'list'> (30,)
<class 'list'> (31,)
<class 'list'> (26,)
<class 'list'> (33,)
<class 'list'> (39,)
<class 'list'> (27,)
<class 'list'> (30,)
<class 'list'> (27,)
<class 'list'> (24,)
<class 'list'> (24,)
<class 'list'> (26,)
<class 'list'> (36,)
<class 'list'> (29,)
<class 'list'> (29,)
<class 'list'> (29,)
<class 'list'> (24,)
<class 'list'> (33,)
<class 'list'> (38,)
<class 'list'> (26,)
<class 'list'> (30,)
<class 'list'> (27,)
<class 'list'> (31,)
<class 'list'> (26,)
<class 'list'> (26,)
<class 'list'> (24,)
<class 'list'> (24,)
<class 'list'> (33,)
<class 'list'> (34,)
<class 'list'> (31,)
<class 'list'> (32,)
<class 'list'> (25,)
<class 'list'> (32,)
<class 'list'> (23,)
<class 'list'> (26,)
<class 'list'> (30,)
<class 'list'> (35,)
<class 'list'> (25,)
<class 'list'> (28,)
<class 'list'> (24,)
<class 'list'> (29,)
<class 'list'> (25,)
<class 'list'> (39,)
<class 'list'> (23,)
<class 'list'> (27,)
<class 'list'

<class 'list'> (31,)
<class 'list'> (33,)
<class 'list'> (29,)
<class 'list'> (30,)
<class 'list'> (30,)
<class 'list'> (33,)
<class 'list'> (37,)
<class 'list'> (25,)
<class 'list'> (27,)
<class 'list'> (39,)
<class 'list'> (32,)
<class 'list'> (27,)
<class 'list'> (24,)
<class 'list'> (24,)
<class 'list'> (29,)
<class 'list'> (25,)
<class 'list'> (24,)
<class 'list'> (24,)
<class 'list'> (34,)
<class 'list'> (38,)
<class 'list'> (36,)
<class 'list'> (28,)
<class 'list'> (25,)
<class 'list'> (27,)
<class 'list'> (27,)
<class 'list'> (28,)
<class 'list'> (30,)
<class 'list'> (38,)
<class 'list'> (35,)
<class 'list'> (30,)
<class 'list'> (26,)
<class 'list'> (35,)
<class 'list'> (26,)
<class 'list'> (32,)
<class 'list'> (38,)
<class 'list'> (39,)
<class 'list'> (23,)
<class 'list'> (28,)
<class 'list'> (35,)
<class 'list'> (27,)
<class 'list'> (28,)
<class 'list'> (30,)
<class 'list'> (27,)
<class 'list'> (35,)
<class 'list'> (25,)
<class 'list'> (28,)
<class 'list'> (23,)
<class 'list'

<class 'list'> (32,)
<class 'list'> (39,)
<class 'list'> (44,)
<class 'list'> (33,)
<class 'list'> (38,)
<class 'list'> (38,)
<class 'list'> (35,)
<class 'list'> (42,)
<class 'list'> (37,)
<class 'list'> (40,)
<class 'list'> (38,)
<class 'list'> (29,)
<class 'list'> (31,)
<class 'list'> (39,)
<class 'list'> (36,)
<class 'list'> (29,)
<class 'list'> (38,)
<class 'list'> (35,)
<class 'list'> (37,)
<class 'list'> (37,)
<class 'list'> (37,)
<class 'list'> (34,)
<class 'list'> (31,)
<class 'list'> (30,)
<class 'list'> (34,)
<class 'list'> (37,)
<class 'list'> (37,)
<class 'list'> (38,)
<class 'list'> (46,)
<class 'list'> (50,)
<class 'list'> (39,)
<class 'list'> (38,)
<class 'list'> (34,)
<class 'list'> (42,)
<class 'list'> (33,)
<class 'list'> (38,)
<class 'list'> (40,)
<class 'list'> (42,)
<class 'list'> (45,)
<class 'list'> (38,)
<class 'list'> (39,)
<class 'list'> (36,)
<class 'list'> (45,)
<class 'list'> (39,)
<class 'list'> (34,)
<class 'list'> (34,)
<class 'list'> (39,)
<class 'list'

In [28]:
# Get a list of subsets that performed best on validation data
maxValAccSubsetIndicies = [index for index in range(len(validationAccuracyList)) if validationAccuracyList[index] == max(validationAccuracyList)]
maxValIndividuals = [individualList[index] for index in maxValAccSubsetIndicies]
maxValSubsets = [[list(allFeatures)[index] for index in range(len(individual)) if individual[index] == 1] for individual in maxValIndividuals]


In [29]:
print('\n---Optimal Feature Subset(s)---\n')
for index in range(len(maxValAccSubsetIndicies)):
    print('Percentile: \t\t\t' + str(percentileList[maxValAccSubsetIndicies[index]]))
    print('Validation Accuracy: \t\t' + str(validationAccuracyList[maxValAccSubsetIndicies[index]]))
    print('Individual: \t' + str(maxValIndividuals[index]))
    print('Number Features In Subset: \t' + str(len(maxValSubsets[index])))
    print('Feature Subset: ' + str(maxValSubsets[index]))



---Optimal Feature Subset(s)---

Percentile: 			0.8051470588235294
Validation Accuracy: 		0.9054771936252061
Individual: 	[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0]
Number Features In Subset: 	47
Feature Subset: [' Source Port', ' Protocol', ' Flow Duration', ' Total Fwd Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', 'Bwd Packet Length Max', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Max', 'Fwd IAT Total', ' Fwd IAT Max', ' Fwd IAT Min', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', 'FI